**Setting up the network**

In [122]:
import pandas as pd

import os

import pypsa

In [123]:

# Specify the path to the directory you want to set as the working directory
new_directory = '/Users/danialriaz/PyPSA-Eur'

# Change the current working directory to the new directory
os.chdir(new_directory)

n = pypsa.Network("results/test-elec/networks/elec_s_8_ec_lcopt_Co2L-1H.nc")


INFO:pypsa.io:Imported network elec_s_8_ec_lcopt_Co2L-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, stores


**Load at each hour (MWh)**

In [124]:
# Transform n.buses_t.marginal_price into a DataFrame
loads_df = pd.DataFrame(n.loads_t.p_set)

loads_df.reset_index(inplace=True)

# Rename the 'index' column to 'timestamp'
loads_df.rename(columns={'snapshot': 'timestamp'}, inplace=True)

# Melt the DataFrame
loads_long = pd.melt(loads_df, id_vars=['timestamp'], var_name='country_bus', value_name='load')

# Split the values in the 'technology' column
loads_long[['country', 'bus']] = loads_long['country_bus'].str.split(' ', expand=True)

# Drop the original 'technology' column
loads_long.drop(columns=['country_bus'], inplace=True)

# Edit country names
loads_long['country'] = loads_long['country'].str[:-1]

loads_long.head(5)


,timestamp,load,country,bus
0,2013-01-01 00:00:00,16318.214789,DE,0
1,2013-01-01 01:00:00,15659.301070,DE,0
2,2013-01-01 02:00:00,15033.356223,DE,0
3,2013-01-01 03:00:00,14801.645166,DE,0
4,2013-01-01 04:00:00,14545.787894,DE,0


In [125]:
# n.loads_t.p.filter(like='DE').sum().sum()
 
loads_grouped = loads_long.groupby(['country'])['load'].sum().reset_index()
loads_grouped['load'] *= (1/1000)  # Convert MWh to GWh
loads_grouped.rename(columns={'load': 'load (GWh)'}, inplace=True)  # Rename the column to indicate GWh
loads_grouped['load (GWh)'] = loads_grouped['load (GWh)']
loads_grouped.round(1)


,country,load (GWh)
0,DE,509158.9
1,FR,492192.9
2,NO,128144.0
3,SE,139580.3


**Price at each hour (€/MWh)**

Mean price per country
Graph per country (excluding the peak price)
Explore: what causes the price hike in Jan?

In [126]:
prices_df = pd.DataFrame(n.buses_t.marginal_price)
prices_df.reset_index(inplace=True)
prices_df.rename(columns={'snapshot': 'timestamp'}, inplace=True)
# Filter out columns containing 'H2' in their names
prices_df = prices_df[[col for col in prices_df.columns if 'H2' not in col]]
prices_long = pd.melt(prices_df, id_vars=['timestamp'], var_name='country_bus', value_name='marginal_price')
prices_long[['country', 'bus']] = prices_long['country_bus'].str.split(' ', expand=True)
prices_long.drop(columns=['country_bus'], inplace=True)
prices_long['country'] = prices_long['country'].str[:-1]

prices_grouped = prices_long.groupby(['country', 'timestamp'])['marginal_price'].mean().reset_index()

prices_long = prices_long[['timestamp','country', 'bus', 'marginal_price']]
prices_long = prices_long.sort_values(by= ['timestamp', 'country', 'bus'])

prices_long.head(10)



,timestamp,country,bus,marginal_price
0,2013-01-01 00:00:00,DE,0,0.025369
8760,2013-01-01 00:00:00,DE,1,0.024368
17520,2013-01-01 00:00:00,DE,2,0.025152
26280,2013-01-01 00:00:00,FR,0,0.024898
35040,2013-01-01 00:00:00,FR,1,0.025161
43800,2013-01-01 00:00:00,FR,2,0.025296
52560,2013-01-01 00:00:00,NO,0,70.430538
61320,2013-01-01 00:00:00,SE,0,0.034816
1,2013-01-01 01:00:00,DE,0,0.025369
8761,2013-01-01 01:00:00,DE,1,0.024368


**Createing Weighted prices**

In [127]:
bus_level_load = pd.merge(prices_long, loads_long, on=['country','bus','timestamp'], how='outer')

bus_level_load = bus_level_load.sort_values(by= ['timestamp', 'country', 'bus'])

# Calculate the load-weighted marginal price
bus_level_load['load_sum'] = bus_level_load.groupby(['timestamp', 'country'])['load'].transform('sum')
bus_level_load['load_weights'] = (bus_level_load['load'] / bus_level_load['load_sum'])
bus_level_load['weighted_price'] = (bus_level_load['load_weights'] * bus_level_load['marginal_price'])

weighted_price = bus_level_load.groupby(['country', 'timestamp'])['weighted_price'].sum().reset_index()
bus_level_load.head(10)
# weighted_price.to_excel('weighted_price.xlsx', index=False)

,timestamp,country,bus,marginal_price,load,load_sum,load_weights,weighted_price
0,2013-01-01 00:00:00,DE,0,0.025369,16318.214789,38617.5824,0.422559,0.010720
8760,2013-01-01 00:00:00,DE,1,0.024368,9646.711803,38617.5824,0.249801,0.006087
17520,2013-01-01 00:00:00,DE,2,0.025152,12652.655808,38617.5824,0.327640,0.008241
26280,2013-01-01 00:00:00,FR,0,0.024898,15125.160211,57243.0000,0.264227,0.006579
35040,2013-01-01 00:00:00,FR,1,0.025161,18520.041682,57243.0000,0.323534,0.008141
43800,2013-01-01 00:00:00,FR,2,0.025296,23597.798107,57243.0000,0.412239,0.010428
52560,2013-01-01 00:00:00,NO,0,70.430538,14741.000000,14741.0000,1.000000,70.430538
61320,2013-01-01 00:00:00,SE,0,0.034816,14610.000000,14610.0000,1.000000,0.034816
1,2013-01-01 01:00:00,DE,0,0.025369,15659.301070,37058.2418,0.422559,0.010720
8761,2013-01-01 01:00:00,DE,1,0.024368,9257.186917,37058.2418,0.249801,0.006087


In [128]:
weighted_price.head(10)

,country,timestamp,weighted_price
0,DE,2013-01-01 00:00:00,0.025048
1,DE,2013-01-01 01:00:00,0.025047
2,DE,2013-01-01 02:00:00,0.025046
3,DE,2013-01-01 03:00:00,0.025046
4,DE,2013-01-01 04:00:00,0.025046
5,DE,2013-01-01 05:00:00,0.025051
6,DE,2013-01-01 06:00:00,0.025186
7,DE,2013-01-01 07:00:00,0.025371
8,DE,2013-01-01 08:00:00,0.025959
9,DE,2013-01-01 09:00:00,0.025669


**Generation at each hour of dispatch and storage units (MWh)**

In [129]:
## GENERATORS ##

# Convert n.generators_t.p into a DataFrame
generators_df = n.generators_t.p.reset_index()

# Rename the 'index' column to 'timestamp'
generators_df.rename(columns={'snapshot': 'timestamp'}, inplace=True)

# Melt the DataFrame
generators_df_long = pd.melt(generators_df, id_vars=['timestamp'], var_name='bus_tech', value_name='generation')

# # Split the values in the 'technology' column
generators_df_long[['country', 'bus', 'technology']] = generators_df_long['bus_tech'].str.split(' ', expand=True)

# # Drop the original 'technology' column
generators_df_long.drop(columns=['bus_tech'], inplace=True)

# # Assuming df is your DataFrame
generators_df_long['country'] = generators_df_long['country'].str[:-1]

# Replace 'technology' values containing 'wind' with 'wind'
generators_df_long.loc[generators_df_long['technology'].str.contains('wind'), 'technology'] = 'wind'


## STORAGE ##

# # Convert n.storage_units_t.p into a DataFrame
# hydro_gen_df = n.storage_units_t.p_dispatch.reset_index()

# # Rename the 'index' column to 'timestamp'
# hydro_gen_df.rename(columns={'snapshot': 'timestamp'}, inplace=True)

# # Melt the DataFrame
# hydro_gen_df_long = pd.melt(hydro_gen_df, id_vars=['timestamp'], var_name='bus_tech', value_name='generation')

# # # Split the values in the 'technology' column
# hydro_gen_df_long[['country', 'bus', 'technology']] = hydro_gen_df_long['bus_tech'].str.split(' ', expand=True)

# # # Drop the original 'technology' column
# hydro_gen_df_long.drop(columns=['bus_tech'], inplace=True)

# # # Assuming df is your DataFrame
# hydro_gen_df_long['country'] = hydro_gen_df_long['country'].str[:-1]

# hydro_gen_df_long['technology'] = hydro_gen_df_long['technology'].replace({'hydro': 'reservoir'})


## MERGE ##

generation_df = generators_df_long.copy()



generation_df.head(10)

,timestamp,generation,country,bus,technology
0,2013-01-01 00:00:00,0.001153,DE,0,CCGT
1,2013-01-01 01:00:00,0.001149,DE,0,CCGT
2,2013-01-01 02:00:00,0.001145,DE,0,CCGT
3,2013-01-01 03:00:00,0.001146,DE,0,CCGT
4,2013-01-01 04:00:00,0.001143,DE,0,CCGT
5,2013-01-01 05:00:00,0.001140,DE,0,CCGT
6,2013-01-01 06:00:00,0.001138,DE,0,CCGT
7,2013-01-01 07:00:00,0.001137,DE,0,CCGT
8,2013-01-01 08:00:00,0.001134,DE,0,CCGT
9,2013-01-01 09:00:00,0.001143,DE,0,CCGT


**Generation by country**

In [130]:
gen_grouped = generation_df.groupby(['country'])['generation'].sum().reset_index()
# gen_grouped['generation'] *= (1/1000)  # Convert MWh to GWh
# gen_grouped.rename(columns={'generation': 'generation (GWh)'}, inplace=True)  # Rename the column to indicate GWh
# gen_grouped['generation (GWh)'] = gen_grouped['generation (GWh)']
gen_grouped.round(1)


,country,generation
0,DE,462905588.7
1,FR,529092910.3
2,NO,119203569.9
3,SE,200062284.6


**Revenue by bus: Merge generation and prices**

In [131]:
revenue_df = pd.merge(generation_df, prices_long, on=['timestamp', 'country','bus'], how='outer')
revenue_df['revenue'] = (revenue_df['generation'] * revenue_df['marginal_price'])
revenue_df = revenue_df[['timestamp', 'country', 'bus', 'technology', 'generation', 'marginal_price', 'revenue']]
revenue_df.head(10)

,timestamp,country,bus,technology,generation,marginal_price,revenue
0,2013-01-01,DE,0,CCGT,0.001153,0.025369,0.000029
1,2013-01-01,DE,0,OCGT,0.000611,0.025369,0.000016
2,2013-01-01,DE,0,coal,0.000808,0.025369,0.000020
3,2013-01-01,DE,0,lignite,0.000977,0.025369,0.000025
4,2013-01-01,DE,0,nuclear,0.001816,0.025369,0.000046
5,2013-01-01,DE,0,wind,1254.051663,0.025369,31.814461
6,2013-01-01,DE,0,wind,61.995995,0.025369,1.572797
7,2013-01-01,DE,0,wind,6883.953744,0.025369,174.641352
8,2013-01-01,DE,0,solar,0.000000,0.025369,0.000000
9,2013-01-01,DE,1,CCGT,0.001119,0.024368,0.000027


**Base Price**

In [132]:
base_price = weighted_price.groupby(['country'])['weighted_price'].mean().reset_index()
base_price.rename(columns={'weighted_price': 'base_price'}, inplace=True)
base_price

,country,base_price
0,DE,62.828979
1,FR,59.653771
2,NO,62.595304
3,SE,57.465326


**Market Value and Value Factor by technology**

In [133]:
mv_df = revenue_df.groupby(['country', 'technology'])[['generation', 'revenue']].sum().reset_index()
mv_df['market_value'] = (mv_df['revenue'] / mv_df['generation'])
mv_df = mv_df[mv_df['generation'] != 0]

mv_vf_df = pd.merge(base_price, mv_df, on=['country'], how='outer')
mv_vf_df['value_factor'] = (mv_vf_df['market_value'] / mv_vf_df['base_price'])

## ADD COUNTRY LEVEL generation %s ##

mv_vf_df['country_sum_gen'] = mv_vf_df.groupby('country')['generation'].transform('sum')
mv_vf_df['gen_perc_mix(%)'] = (mv_vf_df['generation']/mv_vf_df['country_sum_gen']) * 100
mv_vf_df.drop(columns=['country_sum_gen'], inplace=True)
mv_vf_df['gen_perc_mix(%)'] = mv_vf_df['gen_perc_mix(%)'].round(1)
mv_vf_df = mv_vf_df.sort_values(by= ['country', 'gen_perc_mix(%)'], ascending=False)

mv_vf_df.head(5)

,country,base_price,technology,generation,revenue,market_value,value_factor,gen_perc_mix(%)
27,SE,57.465326,wind,1.123234e+08,6.550505e+09,58.318244,1.014842,56.1
26,SE,57.465326,solar,5.961135e+07,1.116859e+08,1.873568,0.032603,29.8
21,SE,57.465326,CCGT,2.682343e+07,2.582369e+09,96.272870,1.675321,13.4
22,SE,57.465326,OCGT,1.304020e+06,1.135302e+09,870.617190,15.150304,0.7
23,SE,57.465326,coal,1.913612e+01,2.212326e+03,115.609980,2.011822,0.0


**Variable Costs and AFC of all technologies (inlc. PHS) (€/MWh)**

In [134]:
## VC ##

# Convert n.generators_t.p into a DataFrame
mc_data = n.generators.marginal_cost.reset_index()

# # Split the values in the 'technology' column
mc_data[['country', 'bus', 'technology']] = mc_data['Generator'].str.split(' ', expand=True)

# # Drop the original 'technology' column
mc_data.drop(columns=['Generator'], inplace=True)

# # Assuming df is your DataFrame
mc_data['country'] = mc_data['country'].str[:-1]

# Replace 'technology' values containing 'wind' with 'wind'
mc_data.loc[mc_data['technology'].str.contains('wind'), 'technology'] = 'wind'

# group bus level data
mc_data_tech = mc_data.groupby(['country', 'technology'])['marginal_cost'].mean().reset_index()
mc_data_tech = mc_data_tech.sort_values(by= ['country', 'marginal_cost','technology'], ascending=False)

# # Convert n.generators_t.p into a DataFrame
# mc_data_st = n.storage_units.marginal_cost.reset_index()

# # # Split the values in the 'technology' column
# mc_data_st[['country', 'bus', 'technology']] = mc_data_st['StorageUnit'].str.split(' ', expand=True)

# # # Drop the original 'technology' column
# mc_data_st.drop(columns=['StorageUnit'], inplace=True)

# # # Assuming df is your DataFrame
# mc_data_st['country'] = mc_data_st['country'].str[:-1]

# # Replace 'technology' values containing 'wind' with 'wind'
# mc_data_st.loc[mc_data_st['technology'].str.contains('wind'), 'technology'] = 'wind'

# # group bus level data
# mc_data_tech_st = mc_data_st.groupby(['country', 'technology'])['marginal_cost'].mean().reset_index()
# mc_data_tech_st = mc_data_tech_st.sort_values(by= ['country', 'marginal_cost','technology'], ascending=False)

marg_cost_df = mc_data_tech.copy()
marg_cost_df = marg_cost_df.sort_values(by= ['country', 'marginal_cost'], ascending=False)

marg_cost_df.rename(columns={'marginal_cost': 'marginal_cost (€/MWh)'}, inplace=True)

## AFC ##

# Convert n.generators_t.p into a DataFrame
afc_data = n.generators.capital_cost.reset_index()

# # Split the values in the 'technology' column
afc_data[['country', 'bus', 'technology']] = afc_data['Generator'].str.split(' ', expand=True)

# # Drop the original 'technology' column
afc_data.drop(columns=['Generator'], inplace=True)

# # Assuming df is your DataFrame
afc_data['country'] = afc_data['country'].str[:-1]

# Replace 'technology' values containing 'wind' with 'wind'
afc_data.loc[mc_data['technology'].str.contains('wind'), 'technology'] = 'wind'

# group bus level data
afc_data_tech = afc_data.groupby(['country', 'technology'])['capital_cost'].mean().reset_index()
afc_data_tech = afc_data_tech.sort_values(by= ['country', 'capital_cost','technology'], ascending=False)
afc_data_tech['capital_cost'] = afc_data_tech['capital_cost']


# # Convert n.generators_t.p into a DataFrame
# afc_data_st = n.storage_units.capital_cost.reset_index()

# # # Split the values in the 'technology' column
# afc_data_st[['country', 'bus', 'technology']] = afc_data_st['StorageUnit'].str.split(' ', expand=True)

# # # Drop the original 'technology' column
# afc_data_st.drop(columns=['StorageUnit'], inplace=True)

# # # Assuming df is your DataFrame
# afc_data_st['country'] = afc_data_st['country'].str[:-1]

# # group bus level data
# afc_data_tech_st = afc_data_st.groupby(['country', 'technology'])['capital_cost'].mean().reset_index()
# afc_data_tech_st = afc_data_tech_st.sort_values(by= ['country', 'capital_cost','technology'], ascending=False)
# afc_data_tech_st['capital_cost'] = afc_data_tech_st['capital_cost']

afc_df = afc_data_tech.copy()
afc_df = afc_df.sort_values(by= ['country', 'capital_cost'], ascending=False)

afc_df.rename(columns={'capital_cost': 'AFC (€/MW)'}, inplace=True)

costs_df = pd.merge(marg_cost_df, afc_df, on=['country','technology'], how='outer')

costs_df['technology'] = costs_df['technology'].replace({'hydro': 'reservoir'})

costs_df.head(5).round(2)

,country,technology,marginal_cost (€/MWh),AFC (€/MW)
0,DE,CCGT,70.43,119304.52
1,DE,OCGT,144.45,62163.31
2,DE,coal,106.46,132163.71
3,DE,lignite,87.98,193840.11
4,DE,nuclear,13.94,526254.83


**Optimal Capacity (GW)**

In [135]:
opt_df = n.generators.p_nom_opt.reset_index()

opt_df[['country', 'bus', 'technology']] = opt_df['Generator'].str.split(' ', expand=True)

opt_df.drop(columns=['Generator'], inplace=True)

opt_df['country'] = opt_df['country'].str[:-1]

opt_df.rename(columns={'p_nom_opt': 'capacity'}, inplace=True)

opt_df.loc[opt_df['technology'].str.contains('wind'), 'technology'] = 'wind'

opt_df_tech = opt_df.groupby(['country','technology'])['capacity'].sum().reset_index()

opt_df_tech['capacity'] = opt_df_tech['capacity']

# storage_cap_df = n.storage_units.p_nom.reset_index()
# storage_cap_df[['country', 'bus', 'technology']] = storage_cap_df['StorageUnit'].str.split(' ', expand=True)
# storage_cap_df.drop(columns=['StorageUnit'], inplace=True)
# storage_cap_df['country'] = storage_cap_df['country'].str[:-1]
# storage_cap_df.rename(columns={'p_nom':'capacity'}, inplace=True)
# storage_cap_df = storage_cap_df.groupby(['country','technology'])['capacity'].sum().reset_index()


cap_df = opt_df_tech.copy()

cap_df['technology'] = cap_df['technology'].replace({'hydro': 'reservoir'})
cap_df['capacity'] /= 1000

cap_df.rename(columns={'capacity': 'capacity (GW)'}, inplace=True)

## Add country wise capacity percentages

cap_df['Country_Sum_Capacity'] = cap_df.groupby('country')['capacity (GW)'].transform('sum')
cap_df['capacity_perc_mix(%)'] = (cap_df['capacity (GW)'] / cap_df['Country_Sum_Capacity']) * 100
cap_df.drop(columns=['Country_Sum_Capacity'], inplace=True)
cap_df['capacity_perc_mix(%)'] = cap_df['capacity_perc_mix(%)'].round(1)
cap_df = cap_df.sort_values(by=['country', 'capacity_perc_mix(%)'], ascending=False)

cap_df.head(5)

,country,technology,capacity (GW),capacity_perc_mix(%)
26,SE,solar,61.603527,48.9
27,SE,wind,43.423798,34.5
22,SE,OCGT,15.121914,12.0
21,SE,CCGT,5.810179,4.6
23,SE,coal,0.000009,0.0


**Calculating Profits**

In [136]:
esm_df = pd.merge(mv_vf_df, costs_df, on=['country','technology'], how='outer')
esm_df = pd.merge(esm_df, cap_df, on=['country','technology'], how='outer')

esm_df['ST profit (€ mil)'] = ((esm_df['revenue'] - (esm_df['marginal_cost (€/MWh)']*esm_df['generation']))/ 1000000)
esm_df['total FOC (€ mil)'] = (((esm_df['AFC (€/MW)']* esm_df['capacity (GW)']*1000))/1000000)
esm_df['LT profit (€ mil)'] = (esm_df['ST profit (€ mil)'] - esm_df['total FOC (€ mil)'])

esm_df = esm_df[esm_df['capacity (GW)'] != 0]

esm_df.head(5)

,country,base_price,technology,generation,revenue,market_value,value_factor,gen_perc_mix(%),marginal_cost (€/MWh),AFC (€/MW),capacity (GW),capacity_perc_mix(%),ST profit (€ mil),total FOC (€ mil),LT profit (€ mil)
0,DE,62.828979,CCGT,1.072429e+08,1.225476e+10,114.271087,1.818764,23.2,70.430442,119304.517221,39.408473,14.0,4701.598219,4701.608905,-0.010686
1,DE,62.828979,OCGT,9.662911e+06,3.029164e+09,313.483581,4.989474,2.1,144.450572,62163.310332,26.815301,9.6,1633.350858,1666.927870,-33.577012
2,DE,62.828979,coal,5.958768e+01,6.771011e+03,113.631054,1.808577,0.0,106.464636,132163.708310,0.000027,0.0,0.000427,0.003613,-0.003186
3,DE,62.828979,lignite,7.085937e+01,6.589365e+03,92.992140,1.480084,0.0,87.979125,193840.105522,0.000019,0.0,0.000355,0.003686,-0.003331
4,DE,62.828979,nuclear,8.437772e+01,6.287817e+03,74.519871,1.186075,0.0,13.938693,526254.833242,0.000015,0.0,0.005112,0.007678,-0.002567


**Compute FLH**

In [137]:
gen_df = generation_df.groupby(['country', 'technology', 'timestamp'])['generation'].sum().reset_index()

# Group by 'country' and 'technology', then count non-zero values in 'generation'
counts = gen_df.groupby(['country', 'technology'])['generation'].agg(lambda x: (x > 10e-1).sum()).reset_index()

# Rename the aggregated column
counts.rename(columns={'generation': 'FLH'}, inplace=True)

counts_sorted = counts.sort_values(by=['country', 'FLH'], ascending=False)
counts_sorted

# Definition taken: Does the technology generate at least 1MWh (10e-1) in one hour in that country

,country,technology,FLH
27,SE,wind,8182
21,SE,CCGT,5414
26,SE,solar,4011
22,SE,OCGT,281
23,SE,coal,0
24,SE,lignite,0
25,SE,nuclear,0
20,NO,wind,8196
14,NO,CCGT,6197
19,NO,solar,4123


Alternate approach to LCOE
![Alt text](image.png)

In [138]:
esm_df = pd.merge(esm_df, counts, on=['country','technology'], how='outer')

esm_df['total VOC (€ mil)'] = ((esm_df['generation'] * esm_df['marginal_cost (€/MWh)'])/1e6)

esm_df['LCOE'] = ((esm_df['total FOC (€ mil)']*1e6 + esm_df['total VOC (€ mil)']) / esm_df['generation'])

esm_df['LCOE 2'] = (esm_df['AFC (€/MW)'] / esm_df['FLH'])

esm_df = esm_df.sort_values(by=['country', 'LCOE'], ascending=False)

esm_df.round(1)

,country,base_price,technology,generation,revenue,market_value,value_factor,gen_perc_mix(%),marginal_cost (€/MWh),AFC (€/MW),capacity (GW),capacity_perc_mix(%),ST profit (€ mil),total FOC (€ mil),LT profit (€ mil),FLH,total VOC (€ mil),LCOE,LCOE 2
22,SE,57.5,OCGT,1304020.1,1.135302e+09,870.6,15.2,0.7,149.7,62163.3,15.1,12.0,940.0,940.0,-0.0,281,195.3,720.9,221.2
25,SE,57.5,nuclear,19.5,1.408600e+03,72.1,1.3,0.0,14.0,526254.8,0.0,0.0,0.0,0.0,-0.0,0,0.0,99.4,inf
27,SE,57.5,wind,112323424.2,6.550505e+09,58.3,1.0,56.1,0.0,178845.4,43.4,34.5,6547.7,7766.1,-1218.4,8182,2.8,69.1,21.9
23,SE,57.5,coal,19.1,2.212300e+03,115.6,2.0,0.0,106.4,132163.7,0.0,0.0,0.0,0.0,-0.0,0,0.0,64.6,inf
24,SE,57.5,lignite,17.6,1.645300e+03,93.6,1.6,0.0,95.9,193840.1,0.0,0.0,-0.0,0.0,-0.0,0,0.0,59.6,inf
26,SE,57.5,solar,59611349.0,1.116859e+08,1.9,0.0,29.8,0.0,39296.5,61.6,48.9,110.5,2420.8,-2310.3,4011,1.2,40.6,9.8
21,SE,57.5,CCGT,26823435.0,2.582369e+09,96.3,1.7,13.4,70.4,119304.5,5.8,4.6,693.2,693.2,-0.0,5414,1889.2,25.8,22.0
15,NO,62.6,OCGT,247.7,2.216161e+06,8945.4,142.9,0.0,170.8,62163.3,0.0,0.1,2.2,2.2,-0.0,8,0.0,8780.4,7770.4
18,NO,62.6,nuclear,51.8,4.165000e+03,80.4,1.3,0.0,14.0,526254.8,0.0,0.0,0.0,0.0,-0.0,0,0.0,81.0,inf
20,NO,62.6,wind,23376912.0,1.704420e+09,72.9,1.2,19.6,0.0,199070.1,7.7,12.6,1703.8,1531.8,172.0,8196,0.6,65.5,24.3


In [139]:
esm_df.to_excel('esm-30.xlsx', index=False)

In [140]:
esm_df_filtered = esm_df[(esm_df['technology'] == 'solar' )][['country', 'gen_perc_mix(%)', 'value_factor']]
esm_df_filtered

,country,gen_perc_mix(%),value_factor
26,SE,29.8,0.032603
19,NO,29.9,0.115429
12,FR,30.0,0.174606
5,DE,29.9,0.237100


In [141]:
gen_grouped.round(1)

,country,generation
0,DE,462905588.7
1,FR,529092910.3
2,NO,119203569.9
3,SE,200062284.6
